In [1]:
import pygame, random, time

from pygame.locals import *


#------------------------------------------------------------------------------
#                                                         FUNCIONES
#------------------------------------------------------------------------------

class Collector ():
    x = 0
    y = 0
    speed = 0
      
class Falling ():
    x = 0
    y = 0
    speed = 0

class Juego ():
    """ Clase que contiene los parámetros del juego, 
        incluida la carga de imágenes por cada nivel 
    """

    def __init__(self) :
        #------------------------------------------1. Inicializa dimensiones pantalla
        self.done = False
        self.width = 1000
        self.height = 600
        self.level = 0
        self.levelprev = -1
        #------------------------------------------2. Crea clase collector (recogedor)
        self.collector = Collector()
        self.collector.speed = 0
        #------------------------------------------3. Crea clase falling (robjeto que cae)
        self.falling = Falling()
        self.falling.speed = 2
        #------------------------------------------4. Inicializa score (puntos) , reloj y fonts
        self.score = 0
        self.screen = pygame.display.set_mode((self.width, self.height), 0, 32)

        self.clock = pygame.time.Clock()
        pygame.time.set_timer(USEREVENT + 1, 1000)

        self.font = pygame.font.Font(None, 72)
        #------------------------------------------5. Define las imágenes por cada nivel 
        dicc_level1 = {"background":"alice_bg.jpg",    "collector":"alice.png",      "falling":"drinkme.png"}
        dicc_level2 = {"background":"alice_bg.jpg",    "collector":"MadHatter.png",  "falling":"hat.png"}
        dicc_level3 = {"background":"redcastle.jpg",   "collector":"redqueen.png",   "falling":"cards.png"}
        dicc_level4 = {"background":"whitecastle.jpg", "collector":"whitequeen.png", "falling":"crown.png"}
        dicc_level5 = {"background":"whitecastle.jpg", "collector":"whitequeen.png", "falling":"crown.png"}
        
        #------------------------------------------6. Crea una lista con todos los diccionarios  
        self.list_img =[dicc_level1, dicc_level2, dicc_level3, dicc_level4, dicc_level5]
        
        #------------------------------------------7. Carga y redimensiona las imagenes del nivel 1  
       
        self.load_images(1)
        
    
    def load_images(self, level) :
        """ Carga una imagen de fondo, una imagen de recogedor (collector) 
            y una imagen de objeto_que_cae (falling)
        """
        diccio =  self.list_img[level-1]
        carpeta = ".\\img\\"
        self.bg_img  = pygame.image.load(carpeta + diccio["background"]).convert() 
        self.bg_img = pygame.transform.scale(self.bg_img, (1000, 600))  
        #____________________________________ObjetoqueCae   
        self.fall_img = pygame.image.load(carpeta + diccio["falling"]).convert_alpha() 
        self.fall_img = pygame.transform.scale(self.fall_img, (70,70))
        #____________________________________Recogedor  
        self.coll_img = pygame.image.load(carpeta + diccio["collector"]).convert_alpha()             
        self.coll_img = pygame.transform.scale(self.coll_img, (150,225))

        self.falling.x = random.randrange(0, self.width- 70)     # Coordenada x del objeto que cae
        self.falling.y = -25                                # Coordenada y del objeto que cae

        self.background_x = 0          # Coordenada x del fondo
        self.background_y = 0          # Coordenada y del fondo

        self.collector.x = self.width/2         # Coordenada x del recogedor
        self.collector.y = self.height - 250    # Coordenada y del recogedor


    #---------------- end def load_images
#---------------- end class Juego  

#------------------------------------------------------------------------------
#                                                         FUNCIONES
#------------------------------------------------------------------------------

#----------------------------------------------------------------- Dtermina el nivel y carga escenario
def calc_load_level(juego) :
    """ En funcion de los puntos calcula el nivel y si ha cambiado el 
        nivel, carga las imágenes del nivel""" 
    
    level = 0
    
    if juego.score <= 5 :
        level = 1
    elif 5 < juego.score <= 25:
        level = 2
        juego.falling.speed = 5
    elif 25 < juego.score <= 50:
        level = 3
        juego.falling.speed = 7
    elif 50 < juego.score <= 80:
        level = 4
    elif 80 < juego.score <= 150:
        level = 5
    #-------------------------- Si por puntos pasa de nivel, se cambian los dibujos
    if level != juego.levelprev :
        juego.levelprev = juego.level
        juego.level = level
        juego.load_images(level)

def gameOver (screen, cuando) :
    """ Mensaje de Juego Terminado """ 
    font = pygame.font.Font(None, 72)
    gameover = font.render("Has perdido en ms {}!!!".format(cuando), 1, Color('white'))
    screen.blit(gameover, (300, 300))
    pygame.display.flip()
    time.sleep(3)
    
    
def bucle_evento(juego):
    """ Lee el evento de teclado y cambia la velocidad del recolector """
    juego.done = False
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            juego.done = True
        elif event.type == pygame.KEYDOWN:
            if event.key == pygame.K_RIGHT:
                juego.collector.speed = 10
            if event.key == pygame.K_LEFT:
                juego.collector.speed = -10
            if event.key == pygame.K_ESCAPE:
                juego.done = True
        elif event.type == pygame.KEYUP:
            if event.key == pygame.K_RIGHT or event.key == pygame.K_LEFT:
                juego.collector.speed = 0
    return juego.done

def juega(juego):

    done = False
    #------------------------------------------------------------- Inicio bucle de juego
    while not done:
        
        #---------------------- Cada 60  analiza si els rectángulo del recolector y del falling intersectan
       juego.clock.tick(60)
       alice_rect = juego.coll_img.get_rect()
       alice_rect.top = juego.collector.y
       alice_rect.left = juego.collector.x
       
       falling_rect = juego.fall_img.get_rect()
       falling_rect.top = juego.falling.y
       falling_rect.left = juego.falling.x
     
       #---------------------- Si intersectan aumenta los puntos (score) y resitua el objeto que cae hacia arriba
  
       if alice_rect.colliderect(falling_rect):
            juego.falling.x = random.randrange( 0, juego.width)
            juego.falling.y = -25
            juego.score += 1

       if calc_load_level(juego) == False :      # Calcula si ha cambiado el nivel y carga las imagenes del nivel 
            done = True
       else:
       #------------------------------- Espera evento y ajusta las coordenadas de movimiento
            bucle_evento(juego)
            #------------------------------- aumenta la velocidad del colector, y la del objeto que cae.                  

            juego.collector.x   += juego.collector.speed
            juego.falling.y     += juego.falling.speed        
            #------------------------------- Se sale de la ventana en vertical, Game Over
            if juego.falling.y > juego.height:
                gameOver(juego.screen, juego.falling.y)
                return(True)
               
            #------------------------------- El colector sale por la derecha -> entra por la izquierda
           
            if juego.collector.x > juego.width:
                juego.collector.x = 0
            #------------------------------- El colector sale por la izquierda -> entra por la derecha           
            if juego.collector.x < 0:
                juego.collector.x = juego.width
            #-------------------------------Muestra las imagenes reubicadas               
            juego.screen.blit(juego.bg_img, (0,0))
            juego.screen.blit(juego.coll_img, (juego.collector.x, juego.collector.y))
            juego.screen.blit(juego.fall_img, (juego.falling.x, juego.falling.y))
            score_text = juego.font.render(" Score: {}" .format(juego.score), 1, Color('white'))
            juego.screen.blit(score_text, (juego.width - 300, 20))
            pygame.display.flip()
       #------------------------------- end if calc_load_level(juego) == False 

    #------------------------------------------------------------- fin bucle de juego
    return(False)
         
#------------------------------------------------------------------------------
#                                                         Proceso principal
#------------------------------------------------------------------------------
pygame.init()

juego = Juego()
salir = False
while not salir:
    salir = juega(juego)

pygame.quit()
quit()
#----------------------------------------------------------------- end main()


pygame 2.0.1 (SDL 2.0.14, Python 3.7.1)
Hello from the pygame community. https://www.pygame.org/contribute.html
